In [1]:
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
from fastai.vision.widgets import *

In [2]:
matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

In [4]:
Path.BASE_PATH = path

In [5]:
path.ls()

(#3) [Path('valid'),Path('labels.csv'),Path('train')]

In [6]:
(path/'train').ls()

(#2) [Path('train/7'),Path('train/3')]

In [7]:
# Load the training datasets into variables threes and sevens
threes = (path/'train'/'3').ls()
sevens = (path/'train'/'7').ls()
threes

(#6131) [Path('train/3/49081.png'),Path('train/3/51816.png'),Path('train/3/39566.png'),Path('train/3/24251.png'),Path('train/3/20989.png'),Path('train/3/29013.png'),Path('train/3/58832.png'),Path('train/3/9294.png'),Path('train/3/20023.png'),Path('train/3/36899.png'),Path('train/3/54952.png'),Path('train/3/45687.png'),Path('train/3/3949.png'),Path('train/3/29991.png'),Path('train/3/47090.png'),Path('train/3/10876.png'),Path('train/3/56823.png'),Path('train/3/25601.png'),Path('train/3/37205.png'),Path('train/3/6825.png')...]

In [8]:
stacked_threes = (torch.stack([tensor(Image.open(o)) for o in threes]).float()/255).view(-1, 28*28)
stacked_sevens = (torch.stack([tensor(Image.open(o)) for o in sevens]).float()/255).view(-1, 28*28)
stacked_threes.shape, stacked_sevens.shape


(torch.Size([6131, 784]), torch.Size([6265, 784]))

In [9]:
image_training_set = torch.cat([stacked_threes, stacked_sevens])
# Create a label with 0 for threes and 1 for sevens
image_training_labels = tensor([1]*len(stacked_threes) + [0]*len(stacked_sevens))
image_training_set.shape, image_training_labels.shape

(torch.Size([12396, 784]), torch.Size([12396]))

In [10]:
# Now we need to zip the training set and labels together
image_training_data = list(zip(image_training_set, image_training_labels))
x,y = image_training_data[0]
x.shape, y

(torch.Size([784]), tensor(1))

In [11]:
def init_weights(size, std = 1.0):
    return (torch.randn(size) * std).requires_grad_()

In [ ]:
weights = init_weights((28*28, 1))


In [13]:
bias = init_weights(1)
bias

tensor([0.3472], requires_grad=True)

In [15]:
def linear(xb): return xb@weights + bias
preds = linear(image_training_set)
preds

tensor([[ -4.3320],
        [ -0.2067],
        [ -1.4215],
        ...,
        [  5.2310],
        [-11.2061],
        [ -6.1952]], grad_fn=<AddBackward0>)

In [24]:
corrects = (preds<0.0).float() == image_training_labels
# corrects, corrects.shape
corrects.float().mean().item()

0.4959973096847534

In [25]:
with torch.no_grad(): weights[0] *= 1.0001

In [26]:
preds = linear(image_training_set)
corrects = (preds<0.0).float() == image_training_labels
corrects.float().mean().item()

0.4959973096847534

In [39]:
weights.shape, preds.shape, corrects.shape



(torch.Size([784, 1]), torch.Size([12396, 1]), torch.Size([12396, 12396]))

In [49]:
def sigmoid(xb):
    return 1/(1+torch.exp(-xb))

In [50]:
def mnist_loss(preds, targs):
    predicts = sigmoid(preds)
    return torch.where(targs ==1, 1-preds, preds).mean()